In [3]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import numpy as np7
from numpy.linalg import norm
import psycopg2
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *

with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

now = datetime.now()
date_time = now.strftime("%Y%m%d")

In [4]:
projects = get_useful_projects()
for p in projects:
    print(p['story'])

creek
minotaur
seeknumbers
sudoku
yosenabe
hop
lights


In [5]:
file = "SimilarityDataframe.csv"
if os.path.exists(file):
    df = pd.read_csv(file)
    print("Loaded Similarity Dataframe")
    if len(projects) != df.shape[0]:
        print("Calculating Similarity Dataframe")
        df = get_similarity_dataframe(projects)
        df.to_csv("SimilarityDataframe.csv", index=False)
        print("Loaded Similarity Dataframe")
else:
    print("Calculating Similarity Dataframe")
    df = get_similarity_dataframe(projects)
    df.to_csv("SimilarityDataframe.csv", index=False)

Loaded Similarity Dataframe


In [6]:
df

,stories,problems,embeddings,cos_creek,cos_minotaur,cos_seeknumbers,cos_sudoku,cos_yosenabe,cos_hop,cos_lights
0,creek,The goal of the game is to blacken some cells ...,"[-0.004310643766075373, -0.01542616169899702, ...",1.000,0.805,0.916,0.828,0.855,0.872,0.892
1,minotaur,There is a player in a labyrinth. The goal is ...,"[-0.01143644005060196, -0.010898801498115063, ...",0.805,1.000,0.813,0.752,0.795,0.837,0.807
2,seeknumbers,"Given a square grid, the goal is to construct ...","[0.000838210282381624, -0.0007265608292073011,...",0.916,0.813,1.000,0.838,0.875,0.913,0.901
3,sudoku,The task of this project is to solve a Sudoku ...,"[0.006194239482283592, -0.002773913089185953, ...",0.828,0.752,0.838,1.000,0.902,0.843,0.855
4,yosenabe,The task of this project is to solve the Japan...,"[0.0007079929346218705, -0.010772258043289185,...",0.855,0.795,0.875,0.902,1.000,0.872,0.870
5,hop,The goal of the puzzle is to find a path betwe...,"[0.005379000212997198, -0.005975912790745497, ...",0.872,0.837,0.913,0.843,0.872,1.000,0.894
6,lights,The puzzle involves illuminating a two-dimensi...,"[0.01010151393711567, -0.0018668060656636953, ...",0.892,0.807,0.901,0.855,0.870,0.894,1.000


In [20]:
def get_avg_score(df, story, story1, story2):
    str_cos_s1 = "cos_" + story1
    str_cos_s2 = "cos_" + story2
    cos_1 = df[df["stories"]==story][str_cos_s1].item()
    cos_2 = df[df["stories"]==story][str_cos_s2].item()
    result = round((cos_1 + cos_2 )/2,2)
    return result

In [21]:
res = get_avg_score(df, "sudoku", "creek", "lights")
res

float

In [23]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [28]:
cursor = conn.cursor()
sql = "SELECT file, story, story_1, story_2 FROM panel"
cursor.execute(sql)
panel = cursor.fetchall()
conn.commit()
cursor.close()
df_panel = pd.DataFrame(panel,columns=["file","story","story1","story2"])

In [43]:
def update_cos_sin(conn, file, cos):
    cursor = conn.cursor()
    sql = "UPDATE panel SET cos_sim = " + str(cos) + " WHERE file = '" + file + "'"
    cursor.execute(sql)
    conn.commit()
    cursor.close()

In [44]:
for index, row in df_panel.iterrows():
    cos = get_avg_score(df,row["story"],row["story1"],row["story2"])
    update_cos_sin(conn,row["file"],cos)

In [40]:
conn.commit()
cursor.close()